In [ ]:
import pandas as pd
import os

# 设置 CSV 文件所在的文件夹路径
csv_folder = r"D:\Microsoft VS Code\uwants"

# 初始化一个空的 DataFrame 用于合并
merged_df = pd.DataFrame()

# 遍历文件夹中的所有 CSV 文件
for file in os.listdir(csv_folder):
    if file.endswith('.csv'):  # 确保只处理 CSV 文件
        file_path = os.path.join(csv_folder, file)
        
        # 使用 UTF-8-SIG 编码读取文件
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig')
            merged_df = pd.concat([merged_df, df], ignore_index=True)  # 合并到总的 DataFrame 中
        except Exception as e:
            print(f"Error reading file {file}: {e}")
            continue

# 数据清洗和标准化
try:
    # 基于 URL 去除重复值
    if '帖子URL' in merged_df.columns:  # 确保列存在
        merged_df = merged_df.drop_duplicates(subset='帖子URL', keep='first')  # 保留首次出现的记录
    else:
        print("URL column not found, skipping deduplication.")

    # 基于时间列去除空值
    if '最初發表時間' in merged_df.columns:  # 确保列存在
        merged_df = merged_df.dropna(subset=['最初發表時間'])  # 删除时间为空的行
    else:
        print("Time column not found, skipping removal of rows with null time.")

    # 字段名标准化映射
    column_mapping = {
        '搜索关键词': 'keyword',
        '帖子标题': 'title',
        '帖子内容': 'post_content',
        '帖子URL': 'url',
        '作者': 'username',
        '回复内容': 'replies',
        '最初發表時間': 'post_time'
    }

    # 替换字段名
    merged_df.rename(columns=column_mapping, inplace=True)

    # 删除 "版區" 列（如果存在）
    if '版區' in merged_df.columns:
        merged_df = merged_df.drop(columns=['版區'])

    # 拆分 "回覆/查看" 列为 "reply_number" 和 "view_number"
    if '回覆/查看' in merged_df.columns:
        merged_df[['reply_number', 'view_number']] = merged_df['回覆/查看'].str.split('/', expand=True)
        merged_df['reply_number'] = pd.to_numeric(merged_df['reply_number'], errors='coerce').fillna(0).astype(int)
        merged_df['view_number'] = pd.to_numeric(merged_df['view_number'], errors='coerce').fillna(0).astype(int)
        merged_df = merged_df.drop(columns=['回覆/查看'])  # 删除原始列

    # 新增 "platform" 列，固定值为 "uwants"
    merged_df['platform'] = 'uwants'

    # 清理 post_time 列中的无关文字
    if 'post_time' in merged_df.columns:
        # 移除包含 "只看該作者分享" 的部分
        merged_df['post_time'] = merged_df['post_time'].str.replace(r'只看該作者分享', '', regex=True).str.strip()
        
        # 转换时间格式为 YYYY-MM-DD HH:MM:SS
        merged_df['post_time'] = pd.to_datetime(merged_df['post_time'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

    # 保存最终标准化的文件
    output_file = r'D:\Microsoft VS Code\uwants\final_standardized_file.csv'
    merged_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"Data cleaning and standardization completed and saved to {output_file}")
except Exception as e:
    print(f"Error during data cleaning and standardization: {e}")


这里需要手动将时间那里列名替换成post_time，然后再运行以下代码

In [ ]:
import pandas as pd

# 当前文件路径
input_file = r'D:\Microsoft VS Code\uwants\final_standardized_file.csv'
output_file = r'D:\Microsoft VS Code\uwants\processed_file.csv'

try:
    # 读取文件
    df = pd.read_csv(input_file, encoding='utf-8-sig')

    # 检查是否存在 "post_time" 列
    if 'post_time' in df.columns:
        # 移除 "post_time" 列中的无效文字 "只看該作者分享"
        df['post_time'] = df['post_time'].str.replace(r'只看該作者分享', '', regex=True).str.strip()

        # 删除 "post_time" 列中为空值的行
        df = df.dropna(subset=['post_time'])

        # 将 "post_time" 转换为标准时间格式
        df['post_time'] = pd.to_datetime(df['post_time'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

        # 再次删除无法解析为时间格式的行
        df = df.dropna(subset=['post_time'])

    else:
        print("The column 'post_time' does not exist in the dataset.")

    # 保存处理后的文件
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"Post time column successfully processed and saved to {output_file}")
except Exception as e:
    print(f"Error during processing: {e}")